## Auto differentiation: PyTorch's autograd


One of the key differences between PyTorch and numpy is the inclusion of an auto differentiation framework, which allows you to track the sequence of operations applied to a tensor and compute the derivatives of the output of the sequence of operations with respect to the input. To achieve this, PyTorch builds a computational graph as you apply operations of tensors tracking the inputs that produced an output.

To train a network, we alternate between two steps over and over again until our network converges. First we compute a *forward pass* which propagates the values input into the network to the output. Second, we go back through the graph propagating gradients from the output of the network back to the input. These gradients are computed with respect to a loss function that we wish to minimize. As we pass back through the network propagating gradients we update the weights of the network in the direction of the negative gradient so as to reduce the loss. The gradients are computed using extensive use of the [chain rule](https://en.wikipedia.org/wiki/Chain_rule) in a method called [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) (AD). We break down the gradient of the loss function with respect to a weight in the network into the product of many partial derivatives by the chain rule. The automatic differentiation support in PyTorch does this (almost) transparently.

To be able to compute derivatives of a sequence of operations, we have to tell PyTorch that we want to track the operations applied to the input tensor which enables it to track the operations applied.

We'll take a look at a very simple computational graphy computing the expression $z = ((x + 2) \times 2) / 5$. The computational graph corresponding to this expression looks like this:

![computational graph](./media/computation-graph.png)

Now we can do a forward pass through the computational graph to compute $z$ computing all the intermediate values.

![computational graph - forward pass](./media/computation-graph-forward.png)

If we look at each individual operation in isolation, we can define the partial derivatives of the inputs to the operation with respect to the output. Later we will chain these together to compute $\frac{dz}{dx}$.
 
$$f_+(x_1, x_2) = x_1 + x_2$$
$$f_\times(x_1, x_2) = x_1 \times x_2$$
$$f_/(x_1, x_2) = x_1 / x_2$$

$$\frac{\partial f_+(x_1, x_2)}{\partial x_1} = 1$$
$$\frac{\partial f_+(x_1, x_2)}{\partial x_1} = 1$$

$$\frac{\partial f_\times(x_1, x_2)}{\partial x_1} = x_2$$
$$\frac{\partial f_\times(x_1, x_2)}{\partial x_2} = x_1$$

$$\frac{\partial f_/(x_1, x_2)}{\partial x_1} = \frac{1}{x_2}$$
$$\frac{\partial f_/(x_1, x_2)}{\partial x_2} = x_1$$

Using these, we can compute the derivates along the edges of a path from the output of the network back to the input using the chain rule: $$\frac{dy}{dx} = \frac{dy}{dz} \frac{dz}{dx}$$. As we trace the path from the output, we compute the gradient of the edge, as we pass a node and move onto a new edge, we compute the gradient of the node's output with respect to its input and then multiply that by the incoming gradient (i.e. we're accumulating the gradient along the path, this is like computing the chain rule in little steps). 

![computational graph - backward pass](./media/computation-graph-backward.png)

Note how we've only computed the gradients along the path back to the single input $x$. This is because the other values are constant and so their gradient is of now interest to use. If $x$ were a weight in a neural network, we'd use the gradient to update the weight. 

If we expand the expression and compute it's derivative in one fell swoop we can check the AD result

$$z = ((x + 2) \times 2) / 5 = \frac{2x}{5} + \frac{4}{5}$$
$$\frac{dz}{dx} = \frac{2}{5}$$

Which is exactly what we got using AD. AD is just a way of breaking up the process into little steps which make it amenable to computation like implemented in PyTorch.

In [1]:
import torch
x = torch.tensor(1.0, requires_grad=True)  # We will use a scalar tensor for simplicity here. 
z0 = x * 2
z1 = z0 + 2
z2 = z1 / 5
z2

tensor(0.8000, grad_fn=<DivBackward0>)

Note that `z2` has a `grad_fn` property

In [2]:
z2.grad_fn  # corresponds to dz/dy

This corresponds to the function that computes the derivative of `z2` with respect to `z1`. Additionally, the `grad_fn` also holds a pointer to the previous operation in the sequence of operations used to construct `z`, we can go back through the chain to see how the gradients are transformed. computation graph tracing from the output of our expression `z2` back to the input `x`, one operation at time. Each `grad_fn` takes in the gradient from the operation ahead of it, transforms it, and passes it to the function before it (remember, we're going backward through the computation graph during gradient computation).

In [3]:
# corresponds to dz2/dz1
z2.grad_fn.next_functions[0][0]

In [4]:
# corresponds to dz1/dz0, basically an identity function
z2.grad_fn.next_functions[0][0].next_functions[0][0]

In [5]:
# And the computational graph is complete, there are no further functions
# on the backward chain as we have reach an input node in the computational tree.
# This simply accumulates gradients
z2.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0]

In [6]:
# No more partial derivatives along the chain.
z2.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0].next_functions

()

In [7]:
# We can compute input_grad * dz2/dz1 
input_grad = torch.tensor(1.)
z2.grad_fn(input_grad)[0]

tensor(0.2000)

In [8]:
# And we can compute input_grad * dz1/dz0
z1.grad_fn(input_grad)[0]

tensor(1.)

In [9]:
# And we can compute input_grad * dz0/dx
z0.grad_fn(input_grad)[0]

tensor(2.)

In [10]:
# Combining these we can get input_grad * dz/dx = input_grad * dz/dy * dy/dx 
# (by the chain rule)

z0.grad_fn(z1.grad_fn(z2.grad_fn(input_grad)[0])[0])[0]

tensor(0.4000)

In [11]:
# Or we can do this in a more general way tracing back from the output
# of the expression to the input
grad_fn = z2.grad_fn
grad = torch.tensor(1.)
while True:
    print("before applying grad_fn:", grad)
    print("grad_fn:", grad_fn)
    grad = grad_fn(grad)
    if len(grad) > 0:
        grad = grad[0]
    else:
        break
    print("after applying grad_fn:", grad)
    print()
    grad_fn = grad_fn.next_functions[0][0]

before applying grad_fn: tensor(1.)
grad_fn: <DivBackward0 object at 0x7f85b81f98d0>
after applying grad_fn: tensor(0.2000)

before applying grad_fn: tensor(0.2000)
grad_fn: <AddBackward0 object at 0x7f858214d810>
after applying grad_fn: tensor(0.2000)

before applying grad_fn: tensor(0.2000)
grad_fn: <MulBackward0 object at 0x7f858214dfd0>
after applying grad_fn: tensor(0.4000)

before applying grad_fn: tensor(0.4000)
grad_fn: <AccumulateGrad object at 0x7f858216b610>
